## Advanced Lane Finding Project

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

---
## Compute the camera calibration using chessboard images

In [1]:
import numpy as np
import importlib
import glob
import cv2
%matplotlib qt
import matplotlib.image as mpimg
import sys
import os
sys.path.append('sources')
import calibration as cb;
# Make a list of calibration images
input_path = "camera_cal/"
images = glob.glob(input_path + 'calibration*.jpg')
chess_calibration_points = cb.Chess_calibration_points(images, [9,6])


## Apply the disturtion to chess images

In [2]:
#apply disturtion coefficients to the chess images
out_path = "output_images/calibration/";
for filename in os.listdir(input_path):
    image = mpimg.imread(  os.path.join( input_path, filename ))    
    undist = chess_calibration_points.undist_image(image);
    cv2.imwrite(os.path.join(out_path , filename), undist, [cv2.IMWRITE_JPEG_QUALITY, 100])

## Apply calibration to the road images and provide the binary images

In [3]:
import binary_outputs as bo
import numpy as np
import cv2
import matplotlib.pyplot as plt
import image_processing as im

input_path  = './test_images/'
#apply disturtion coefficients to the chess images
out_path = "output_images/binaries/";
for filename in os.listdir(input_path):
    image = mpimg.imread(  os.path.join( input_path, filename ))    
    undist = chess_calibration_points.undist_image(image);
    binary = im.get_binary(undist)
    cv2.imwrite(os.path.join(out_path , filename), bo.to_binary_image(binary), [cv2.IMWRITE_JPEG_QUALITY, 100])


## Apply a perspective transform to rectify binary image ("birds-eye view").

In [4]:
import sys
import os
sys.path.append('sources')
import transformation as tr
import matplotlib.pyplot as plt
%matplotlib inline


def get_model_polygon (img):
    img_size = [img.shape[1], img.shape[0]]
    src = np.float32(
     [[(img_size[0] * 11/ 24), img_size[1] * 5/8],
     [((img_size[0] / 8) ), img_size[1]],
     [(img_size[0] *  11 / 12) , img_size[1]],
     [(img_size[0] * 13/ 24 ), img_size[1] * 5 / 8]])
    dst = np.float32(
     [[(img_size[0] / 6-50), 0],
     [(img_size[0] / 6-50), img_size[1]],
     [(img_size[0] * 5 / 6 -50), img_size[1]],
     [(img_size[0] * 5 / 6-50), 0]])
    return src, dst

input_path  = './test_images/'
out_path = "output_images/bird_eye/";
warper = [];
for filename in os.listdir(input_path):
    image = mpimg.imread(  os.path.join( input_path, filename ))    
    undist = chess_calibration_points.undist_image(image);
    if not warper:
        src, dst = get_model_polygon(image)
        warper = tr.ImageWarper( src, dst);
    rgbImg = cv2.cvtColor(undist, cv2.COLOR_BGR2RGB)            
    wrap_img = warper.warp_image(rgbImg);        
    cv2.polylines(wrap_img, [dst.astype(int)], True, (0,255,0), 2)
    cv2.imwrite(os.path.join(out_path , filename), wrap_img, [cv2.IMWRITE_JPEG_QUALITY, 100])    
    


## Apply the same transformation for the binary images

In [5]:
out_path = "output_images/bird_eye_binaries"

for filename in os.listdir(input_path):
    image = mpimg.imread(  os.path.join( input_path, filename ))    
    undist = chess_calibration_points.undist_image(image);
    binary = get_binary(undist)
    wrap_img = warper.warp_image(binary)
    cv2.imwrite(os.path.join(out_path , filename), bo.to_binary_image(wrap_img), [cv2.IMWRITE_JPEG_QUALITY, 100])

## Detect lane pixels and fit to find the lane boundary.

In [6]:
import lane_detection as ld
import matplotlib.pyplot as plt

out_path = "output_images/lane_detection_bird_eye"

for filename in os.listdir(input_path):
    image = mpimg.imread(  os.path.join( input_path, filename ))    
    undist = chess_calibration_points.undist_image(image);
    binary = get_binary(undist)
    wrap_img = warper.warp_image(binary)
    # View your output
    lane_detector = ld.LaneDetector()
    lane_detector.find_next_poly(wrap_img)
    result = lane_detector.draw_the_lines(wrap_img)
    cv2.imwrite(os.path.join(out_path , filename), result, [cv2.IMWRITE_JPEG_QUALITY, 100])


## Provide full lane detection for the all test images

In [7]:
import lane_detection as ld
import matplotlib.pyplot as plt

out_path = "output_images/lane_detection"

for filename in os.listdir(input_path):
    image_processor = im.ImageProcessor(chess_calibration_points);
    print(filename)
    image = mpimg.imread(  os.path.join( input_path, filename ))  
    result = image_processor.process(image, True);
    rgbImg = cv2.cvtColor(result, cv2.COLOR_BGR2RGB) 
    cv2.imwrite(os.path.join(out_path , filename), rgbImg, [cv2.IMWRITE_JPEG_QUALITY, 100])

test6.jpg
Curvature  (9885.845315483477, 5952.7378399521695)
Center offset  -0.10312500000000001  meters
straight_lines2.jpg
Curvature  (5628.5874508696452, 3291.8705395992497)
Center offset  -0.24921875000000002  meters
test1.jpg
Curvature  (3716.08862140163, 451.17333008711148)
Center offset  -0.10742187500000001  meters
test2.jpg
Curvature  (389.94004840731407, 310.18485707542425)
Center offset  0.0171875  meters
straight_lines1.jpg
Curvature  (7789.0980269444108, 1580.0949818703452)
Center offset  -0.275  meters
test3.jpg
Curvature  (487.98428476188928, 543.42047261909431)
Center offset  -0.1375  meters
test5.jpg
Curvature  (654.5643285155345, 332.69874730810972)
Center offset  -0.24921875000000002  meters
test4.jpg
Curvature  (1663.3821351231024, 551.83400606323119)
Center offset  -0.038671875  meters


## Provide lane detection for the video

In [8]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

project_video_output = 'output_videos/project_video.mp4'
clip2 = VideoFileClip('project_video.mp4')
image_processor = im.ImageProcessor(chess_calibration_points);
process_image_lambda = lambda image: image_processor.process(image)
project_clip = clip2.fl_image(process_image_lambda)
%time project_clip.write_videofile(project_video_output, audio=False)
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(project_video_output))

[MoviePy] >>>> Building video output_videos/project_video.mp4
[MoviePy] Writing video output_videos/project_video.mp4


 99%|█████████▉| 125/126 [00:42<00:00,  2.83it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: output_videos/project_video.mp4 

CPU times: user 24.6 s, sys: 6.69 s, total: 31.3 s
Wall time: 45.9 s


## Lane detection for the challenged videos

In [9]:
project_video_output = 'output_videos/challenge_video.mp4'
clip2 = VideoFileClip('challenge_video.mp4')
image_processor = im.ImageProcessor(chess_calibration_points);
process_image_lambda = lambda image: image_processor.process(image) #bo.to_binary_image(get_binary(image)) 
challenge_video_clip = clip2.fl_image(process_image_lambda)
%time challenge_video_clip.write_videofile(project_video_output, audio=False)
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(project_video_output))

[MoviePy] >>>> Building video output_videos/challenge_video.mp4
[MoviePy] Writing video output_videos/challenge_video.mp4


100%|██████████| 485/485 [02:45<00:00,  2.86it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: output_videos/challenge_video.mp4 

CPU times: user 1min 32s, sys: 26 s, total: 1min 58s
Wall time: 2min 48s


In [10]:
project_video_output = 'output_videos/harder_challenge_video.mp4'
clip2 = VideoFileClip('harder_challenge_video.mp4')
image_processor = im.ImageProcessor(chess_calibration_points);
process_image_lambda = lambda image: image_processor.process(image) #bo.to_binary_image(get_binary(image)) 
challenge_video_clip = clip2.fl_image(process_image_lambda)
%time challenge_video_clip.write_videofile(project_video_output, audio=False)
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(project_video_output))

[MoviePy] >>>> Building video output_videos/harder_challenge_video.mp4
[MoviePy] Writing video output_videos/harder_challenge_video.mp4


100%|█████████▉| 1199/1200 [07:41<00:00,  2.73it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: output_videos/harder_challenge_video.mp4 

CPU times: user 4min 12s, sys: 54.6 s, total: 5min 7s
Wall time: 7min 45s
